# Sample Apache Spark Notebook

Here's an example of cleaning and analyzing an Apache access log, but this time within an interactive Notebook environment!

While prototyping data engineering solutions, notebook environments are popular.

If you installed the pyspark package or are working within an already-established environment for Spark, things will probably "just work." But if not, using the findspark package will tie the notebook to your existing Spark installation:

In [1]:
import findspark
findspark.init()

One nice thing about notebooks is that you can leave little comments and explanations like this, in markdown format.

We'll start by importing the stuff we need, and creating a SparkSession.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col, count, desc
from pyspark.sql.types import IntegerType

# Initialize Spark session
spark = SparkSession.builder.appName("ApacheLogAnalysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/11 08:06:07 WARN Utils: Your hostname, bitedFruit.local, resolves to a loopback address: 127.0.0.1; using 192.168.15.3 instead (on interface en0)
25/11/11 08:06:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/11 08:06:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Next we'll load up our sample access log, and load in the raw text into a Dataframe.

In [4]:
# Define log file path (Update this path to your log file location)
log_file = "./access_log.txt"

# Read log file as text
logs_df = spark.read.text(log_file)

We'll now parse the log into the fields we are interested in. Note, we know there is some bad data in here where the status code is empty.

In [5]:
# Regular expression pattern to extract fields
log_pattern = r'(\S+) - - \[(.*?)\] "(.*?)" (\S{3}) (\d+) "(.*?)"'

# Extract fields using regex
parsed_logs_df = logs_df.select(
    regexp_extract('value', log_pattern, 1).alias("ip_address"),
    regexp_extract('value', log_pattern, 2).alias("timestamp"),
    regexp_extract('value', log_pattern, 3).alias("request"),
    regexp_extract('value', log_pattern, 4).alias("status"),
    regexp_extract('value', log_pattern, 5).cast("integer").alias("bytes"),
    regexp_extract('value', log_pattern, 6).alias("user_agent")
)

We'll use a filter to just remove those bogus rows with no status, and cast the remaining status codes to integers.

In [6]:
# Filter out rows with empty status fields
cleaned_df = parsed_logs_df.filter(col("status").isNotNull() & (col("status") != ""))
cleaned_df = cleaned_df.withColumn("status", col("status").cast(IntegerType()))

A nice thing about notebooks is you can break up your processing into these separate blocks, and inspect the output just for whatever it is you're doing. Then you can go back and iterate on that piece of code as needed, rather than re-running everything.

Let's further process our data to split out the method and endpoint from the request field, and then preview the resulting Dataframe we have thusfar:

In [7]:
# Split request field to get HTTP method and endpoint
parsed_logs_df = cleaned_df.withColumn("method", regexp_extract("request", r'(\S+)', 1)) \
                               .withColumn("endpoint", regexp_extract("request", r' (\S+) ', 1))

# Show parsed log data
parsed_logs_df.show(10, truncate=False)

+--------------+--------------------------+----------------------------------+------+-----+----------------------------------+------+---------------------+
|ip_address    |timestamp                 |request                           |status|bytes|user_agent                        |method|endpoint             |
+--------------+--------------------------+----------------------------------+------+-----+----------------------------------+------+---------------------+
|66.249.75.159 |29/Nov/2015:03:50:05 +0000|GET /robots.txt HTTP/1.1          |200   |55   |-                                 |GET   |/robots.txt          |
|66.249.75.168 |29/Nov/2015:03:50:06 +0000|GET /blog/ HTTP/1.1               |200   |8083 |-                                 |GET   |/blog/               |
|185.71.216.232|29/Nov/2015:03:53:15 +0000|POST /wp-login.php HTTP/1.1       |200   |1691 |http://nohatenews.com/wp-login.php|POST  |/wp-login.php        |
|54.165.199.171|29/Nov/2015:04:32:27 +0000|GET /sitemap_index.xm

Now let's start doing some analysis. We'll start with displaying the top 10 IP addresses. Unfortunately, as usual, there's a hacker trying to DOS me:

In [8]:
# 1. Count requests per IP address
ip_count_df = parsed_logs_df.groupBy("ip_address").agg(count("*").alias("request_count")).orderBy(desc("request_count"))
ip_count_df.show(10)

+--------------+-------------+
|    ip_address|request_count|
+--------------+-------------+
| 46.166.139.20|        68487|
|54.165.199.171|         4381|
|195.154.250.88|         1723|
| 97.100.169.53|          240|
| 62.210.88.201|           81|
|  66.249.66.59|           69|
|  66.249.66.62|           54|
|   66.249.66.3|           47|
|   52.91.1.103|           39|
| 172.56.26.235|           24|
+--------------+-------------+
only showing top 10 rows


Let's look at the top endpoints, and right away we can see that our friend is trying to break into my WordPress site through xmlrpc.php vulnerabilities and trying to brute-force their way in:

In [9]:
# 2. Most requested endpoints
endpoint_count_df = parsed_logs_df.groupBy("endpoint").agg(count("*").alias("endpoint_count")).orderBy(desc("endpoint_count"))
endpoint_count_df.show(10)

+--------------------+--------------+
|            endpoint|endpoint_count|
+--------------------+--------------+
|         /xmlrpc.php|         68494|
|       /wp-login.php|          1923|
|                   /|           336|
|              /blog/|           138|
|         /robots.txt|           123|
|   /post-sitemap.xml|           118|
|  /sitemap_index.xml|           118|
|   /page-sitemap.xml|           117|
|/category-sitemap...|           117|
| /orlando-headlines/|            95|
+--------------------+--------------+
only showing top 10 rows


Let's also take a look at the top status codes. Looks like they've succeeded in making my site unstable:

In [10]:
# 3. HTTP status code distribution
status_count_df = parsed_logs_df.groupBy("status").agg(count("*").alias("status_count")).orderBy(desc("status_count"))
status_count_df.show(10)

+------+------------+
|status|status_count|
+------+------------+
|   200|       64965|
|   500|       10714|
|   301|         159|
|   404|          24|
|   400|           2|
|   302|           2|
|   405|           1|
+------+------------+



Finally we'll shut things down:

In [11]:
# Stop Spark session
spark.stop()